In [5]:
import sqlite3
conn = sqlite3.connect("AQI1.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [6]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

['屏東(枋山)', '左營', '線西', '新營', '基隆', '林園', '屏東', '中山', '楠梓', '麥寮', '斗六', '汐止', '松山', '觀音', '新店', '馬公', '高雄（湖內）', '二林', '朴子', '大同', '菜寮', '林口', '善化', '南投', '新北(樹林)', '關山', '淡水', '嘉義', '前鎮', '富貴角', '宜蘭（頭城）', '彰化（員林）', '彰化', '安南', '頭份', '宜蘭', '新莊', '沙鹿', '橋頭', '臺西', '平鎮', '大寮', '西屯', '冬山', '金門', '大里', '馬祖', '湖口', '大城', '三義', '土城', '鳳山', '潮州', '大園', '永和', '豐原', '仁武', '中壢', '板橋', '恆春', '萬華', '新竹', '古亭', '士林', '臺南（麻豆）', '桃園', '三重', '陽明', '新港', '萬里', '埔里', '苗栗', '小港', '臺東', '竹東', '竹山', '前金', '花蓮', '崙背', '忠明', '復興', '臺南', '屏東（琉球）', '美濃', '龍潭']


In [7]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

[['2024-11-04 10:00', '屏東縣', 34, 18, '良好', 22.260899, 120.651472],
 ['2024-10-28 09:00', '屏東縣', 17, 6.2, '良好', 22.260899, 120.651472]]

In [8]:
import sqlite3
conn = sqlite3.connect("AQI1.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [9]:
import sqlite3
conn = sqlite3.connect("AQI1.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))



        

In [10]:
conn = sqlite3.connect("AQI1.db")
with conn:
    cursor = conn.cursor()
    sql = '''
    SELECT DISTINCT sitename
    FROM records
    '''
    cursor.execute(sql)
    sitenames = []

    for items in cursor.fetchall():
        sitenames.append(items[0])
    
    print(sitenames)

['屏東(枋山)', '宜蘭（頭城）', '新北(樹林)', '屏東（琉球）', '臺南（麻豆）', '高雄（湖內）', '彰化（員林）', '大城', '富貴角', '麥寮', '關山', '馬公', '金門', '馬祖', '埔里', '復興', '永和', '竹山', '中壢', '三重', '冬山', '宜蘭', '陽明', '花蓮', '臺東', '恆春', '潮州', '屏東', '小港', '前鎮', '前金', '左營', '楠梓', '林園', '大寮', '鳳山', '仁武', '橋頭', '美濃', '臺南', '安南', '善化', '新營', '嘉義', '臺西', '朴子', '新港', '崙背', '斗六', '南投', '二林', '線西', '彰化', '西屯', '忠明', '大里', '沙鹿', '豐原', '三義', '苗栗', '頭份', '新竹', '竹東', '湖口', '龍潭', '平鎮', '觀音', '大園', '桃園', '大同', '松山', '古亭', '萬華', '中山', '士林', '淡水', '林口', '菜寮', '新莊', '板橋', '土城', '新店', '萬里', '汐止', '基隆']


In [11]:
conn = sqlite3.connect("AQI1.db")
with conn:
    cursor = conn.cursor()        
    sql = '''
    SELECT date,county,aqi,pm25,status,lat,lon
    FROM records
    WHERE sitename=?
    ORDER BY date DESC;
    '''
    cursor.execute(sql,('富貴角',))
    sitename_list = [list(item) for item in cursor.fetchall()]
    print(sitename_list)

[['2024-11-12 14:00', '新北市', 42, 7, '良好', 25.29681695, 121.53656894], ['2024-11-12 13:00', '新北市', 41, 6, '良好', 25.29681695, 121.53656894], ['2024-11-12 13:00', '新北市', 41, 6, '良好', 25.29681695, 121.53656894], ['2024-11-12 12:00', '新北市', 40, 7, '良好', 25.29681695, 121.53656894], ['2024-11-12 12:00', '新北市', 40, 7, '良好', 25.29681695, 121.53656894], ['2024-11-12 11:00', '新北市', 40, 7, '良好', 25.29681695, 121.53656894], ['2024-11-12 11:00', '新北市', 40, 7, '良好', 25.29681695, 121.53656894], ['2024-11-12 10:00', '新北市', 39, 10, '良好', 25.29681695, 121.53656894], ['2024-11-12 10:00', '新北市', 39, 10, '良好', 25.29681695, 121.53656894], ['2024-11-12 09:00', '新北市', 39, 10, '良好', 25.29681695, 121.53656894], ['2024-11-12 09:00', '新北市', 39, 10, '良好', 25.29681695, 121.53656894], ['2024-11-12 08:00', '新北市', 39, 10, '良好', 25.29681695, 121.53656894]]
